<a href="https://colab.research.google.com/github/fcoliveira-utfpr/chuva_probabilidade_sh/blob/main/analises_decendial_chuva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando bibliotecas e autorizando Drive

In [1]:
# Limpa a saída do terminal no Google Colab
from google.colab import output
output.clear()
# Instala ou atualiza a bibliotecas
!pip install --upgrade gspread -q
!pip install pymannkendall -q
# Autentica o usuário no Google Colab
from google.colab import auth
auth.authenticate_user()
# Importa a biblioteca para interagir com Google Sheets
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
#importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from datetime import timedelta
import math
from scipy import stats
from scipy.stats import (gamma, norm, ks_2samp, beta, weibull_min, expon, lognorm)
from matplotlib.ticker import FuncFormatter
from scipy.stats import ksone
from sklearn.preprocessing import MinMaxScaler
import pymannkendall as mk
from scipy.stats import linregress

##Importando e abrindo planilha do Drive

In [10]:
#importantando planilha e abrindo***********************************************
nome_da_planilha = 'dados_chuva_oeste'
nome_da_aba = "df_resultados"
spreasheet = gc.open(nome_da_planilha)
worksheet = spreasheet.worksheet(nome_da_aba)
df = pd.DataFrame(worksheet.get_all_values())
head = df.loc[0].to_list()
df.columns = head
df = df[1:]
cidade = df['Municipio']
df = df.drop(columns=['Municipio'])
df = df.replace({',': '.'}, regex=True)
df = df.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=1)
df['Municipio'] = cidade
df['Ano'] = df['Ano'].astype(int)
df['Mês'] = df['Mês'].astype(int)
df['Dia'] = df['Dia'].astype(int)
#filtra município desejado
municipio = "Santa Helena"
df = df.loc[df['Municipio'] == municipio]
# Cria uma nova coluna 'data' a partir das colunas 'dia', 'mês' e 'ano'
data = df['Ano'].astype(str) + '-' + df['Mês'].astype(str) + '-' + df['Dia'].astype(str)
dia = []
for i in data:
  a = pd.to_datetime(datetime.strptime(i, '%Y-%m-%d').date())
  dia.append(a)
df['Data'] = dia

df

,Dia,Mês,Ano,Chuva (mm),lat,lon,alt,Municipio,Data
317045,1,1,1990,1.4,-24.887224,-54.30623,238.0,Santa Helena,1990-01-01
317046,2,1,1990,19.5,-24.887224,-54.30623,238.0,Santa Helena,1990-01-02
317047,3,1,1990,3.8,-24.887224,-54.30623,238.0,Santa Helena,1990-01-03
317048,4,1,1990,0.0,-24.887224,-54.30623,238.0,Santa Helena,1990-01-04
317049,5,1,1990,0.0,-24.887224,-54.30623,238.0,Santa Helena,1990-01-05
...,...,...,...,...,...,...,...,...,...
334572,27,12,2023,0.0,-24.887224,-54.30623,238.0,Santa Helena,2023-12-27
334573,28,12,2023,0.0,-24.887224,-54.30623,238.0,Santa Helena,2023-12-28
334574,29,12,2023,9.8,-24.887224,-54.30623,238.0,Santa Helena,2023-12-29
334575,30,12,2023,0.0,-24.887224,-54.30623,238.0,Santa Helena,2023-12-30
